## Pluralsight Course : Building and Deploying RAG in Production

#### Demo: Basic RAG

In [ ]:
# install the required libraries
!pip install -qqq llama-index llama-index-llms-openai llama-index-vector-stores-chroma


In [ ]:
import os
# Load API key from environment (optionally from a .env file)
try:
    from dotenv import load_dotenv  # pip install python-dotenv (if not available)
    load_dotenv()
except Exception:
    pass

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError(
        "OPENAI_API_KEY is not set. Export it in your shell or put it in a .env file."
    )

### Define Embedding and LLM Model

In [14]:
from llama_index.embeddings.openai import OpenAIEmbedding
# define embeding model 
embed_model = OpenAIEmbedding()

In [15]:
from llama_index.llms.openai import OpenAI
# define LLM model
llm = OpenAI(model="gpt-4o", temperature=0)

In [17]:
from llama_index.core import Settings

# setting embedding model and llm model globally
Settings.embed_model = embed_model
Settings.llm = llm

### Ingestion Pipeline

In [18]:
# set chunk size
Settings.chunk_size = 1024

In [19]:
# import libraries 
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter

# load
documents = SimpleDirectoryReader("data").load_data()


In [20]:
# define chunking strategy
text_splitter = TokenTextSplitter()

In [21]:
# define vector database and store 
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# create local in-memory client
chroma_client = chromadb.EphemeralClient()
# create a collection
chroma_collection = chroma_client.create_collection("ps-foo-rag", get_or_create=True)
# define the vector store using the collection
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


ValueError: An instance of Chroma already exists for ephemeral with different settings

In [22]:
# define ingestion pipeline
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        embed_model,
    ],
    vector_store=vector_store,
)


In [23]:
# run the ingestion pipeline
nodes = pipeline.run(documents=documents)
print(f"number of nodes or chunks : {len(nodes)}")

Failed to send telemetry event CollectionAddEvent: module 'chromadb' has no attribute 'get_settings'


number of nodes or chunks : 7


### RAG Pipeline

In [24]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model,
)

In [25]:
# create semantic query engine 
vector_query_engine = vector_index.as_query_engine()

In [30]:
# query 
response = vector_query_engine.query("provide a markdown summary of the two men's timeline.")
print(response)

```markdown
### Timeline Summary of Bar and Qux

- **1998**: Bar launches his first venture, Cam, a tech start-up focused on data management.
- **2000**: Bar founds Tres, a company providing cloud-based solutions for small businesses.
- **2009**: Bar begins laying the groundwork for Foo, researching the market and developing a business plan.
- **Early 2010**: Bar officially founds Foo, with Qux joining as co-founder.
- **November 2010**: Foo's website goes live, offering a curated selection of adventure gear.
- **2015**: Foo establishes itself as a leading player in the online adventure gear market.
```



***Sample Queries to test***

- When did Bar started his entrepreneurial journey ?
- Who are the co-founders of the company ?
- What is the Bar's AI vision?
- Who is the CEO of company Foo? and what are the other companies he has started earlier.
